In [319]:
import pandas as pd
import numpy as np

In [320]:
new_file=pd.read_csv('C:/Users/yifeichongtian/Desktop/UCL course material/data analytics/3rd report/new_business.csv')

In [321]:
new_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16273 entries, 0 to 16272
Data columns (total 12 columns):
Unnamed: 0     16273 non-null int64
business_id    16273 non-null object
categories     16273 non-null object
city           16273 non-null object
stars          16273 non-null float64
state          16273 non-null object
Noise          16273 non-null object
Wi-Fi          16273 non-null object
Price          16273 non-null float64
Kids           16273 non-null bool
Take-out       16273 non-null bool
park           16273 non-null bool
dtypes: bool(3), float64(2), int64(1), object(6)
memory usage: 1.2+ MB


## Linear Feature Selection by Pearson Correlation Coefficient

In [322]:
# import sklearn and use its linear regression model
from sklearn import linear_model

In [323]:
features=["Noise","Kids"]
first_file=new_file[features]
first_file["Noise"].value_counts()

average      11472
quiet         3258
loud          1151
very_loud      392
Name: Noise, dtype: int64

In [326]:
first_file.loc[first_file.Noise=='quiet',"Noise":]=0
first_file.loc[first_file.Noise=='average',"Noise":]=1
first_file.loc[first_file.Noise=='loud',"Noise":]=2
first_file.loc[first_file.Noise=='very_loud',"Noise":]=3
first_file["Noise"]=label_encoder.fit_transform(first_file["Noise"])
first_file.Noise.value_counts()

C:\Users\yifeichongtian\Anaconda3\lib\site-packages\pandas\core\indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\yifeichongtian\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1    11472
0     3258
2     1151
3      392
Name: Noise, dtype: int64

In [327]:
first_file["Kids"].value_counts()

True     13015
False     3258
Name: Kids, dtype: int64

In [328]:
from sklearn import preprocessing
np.random.seed(12)

label_encoder=preprocessing.LabelEncoder()

first_file['Kids']=label_encoder.fit_transform(first_file['Kids'])

C:\Users\yifeichongtian\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [329]:
second_file=pd.DataFrame(new_file["Wi-Fi"])
second_file.loc[second_file['Wi-Fi']=="no","Wi-Fi":]=0
second_file.loc[second_file['Wi-Fi']=="free","Wi-Fi":]=1
second_file.loc[second_file['Wi-Fi']=="paid","Wi-Fi":]=2
second_file["Wi-Fi"]=label_encoder.fit_transform(second_file["Wi-Fi"])
second_file["Wi-Fi"].value_counts()

0    9566
1    6578
2     129
Name: Wi-Fi, dtype: int64

In [330]:
new_file["park"].value_counts()

True     12595
False     3678
Name: park, dtype: int64

In [332]:
third_file=pd.DataFrame(new_file["park"])
third_file["park"]=label_encoder.fit_transform(third_file["park"])
third_file.park.value_counts()

1    12595
0     3678
Name: park, dtype: int64

In [333]:
fourth_file=pd.DataFrame(new_file["Take-out"])
fourth_file["Take-out"]=label_encoder.fit_transform(fourth_file["Take-out"])
fourth_file["Take-out"].value_counts()

1    15047
0     1226
Name: Take-out, dtype: int64

In [334]:
final_file=pd.concat([first_file,second_file,third_file, fourth_file, new_file["Price"]],axis=1)

In [335]:
final_file.Noise.value_counts()

1    11472
0     3258
2     1151
3      392
Name: Noise, dtype: int64

In [336]:
final_file.Price.value_counts()

2.0    8188
1.0    7112
3.0     787
4.0     186
Name: Price, dtype: int64

In [337]:
final_file['Take-out'].value_counts()

1    15047
0     1226
Name: Take-out, dtype: int64

In [338]:
final_file.park.value_counts()

1    12595
0     3678
Name: park, dtype: int64

In [339]:
final_file['Wi-Fi'].value_counts()

0    9566
1    6578
2     129
Name: Wi-Fi, dtype: int64

In [340]:
final_file.Kids.value_counts()

1    13015
0     3258
Name: Kids, dtype: int64

In [341]:
x=final_file
y=new_file["stars"]

In [342]:
x.columns

Index(['Noise', 'Kids', 'Wi-Fi', 'park', 'Take-out', 'Price'], dtype='object')

#### *There are 6 features in total, which is not many.*

And we will use Pearson Correlation Coefficient to measure how two set of values are linearly correlated, and the result value must be in the range [-1,1]. If it is equal or nearly equal to 0, it suggests that they are not correlated, but when the value gets close to one, it exhibit positive correlation, and vice verse when it gets close to minus 1.

In [343]:
import math

# Use this function to get R value (not squared)
def computeCorrelation(x, y):
    xBar = np.mean(x)
    yBar = np.mean(y)
    SSR = 0
    varX = 0
    varY = 0
    for i in range(0 , len(x)):
        diffXXBar = x[i] - xBar
        diffYYBar = y[i] - yBar
        SSR += (diffXXBar * diffYYBar)
        varX += diffXXBar**2
        varY += diffYYBar**2
        SST = math.sqrt(varX * varY)
    return SSR / SST

# An empty array for tuples
linear_features_and_R = [] 
# An empty array for selected linear features
linear_features = []
# An array for R values
R = np.zeros(len(final_file.columns))
# For every feature, calculate how much it is correlated with
# 'stars rating'
for i in range(0,len(final_file.columns)):
    testX = final_file[final_file.columns[i]].values
    testY = y.values
    R[i] = computeCorrelation(testX, testY)
    if abs(R[i]) > 0:
        linear_features.append(final_file.columns[i])
        linear_features_and_R.append((final_file.columns[i], R[i]))

print("Here are the selected values and their R values")
for i in range(0, len(linear_features_and_R)):
    print(linear_features_and_R[i][0],":",linear_features_and_R[i][1])

Here are the selected values and their R values
Noise : -0.144149905578
Kids : -0.0566649477546
Wi-Fi : 0.029484947293
park : 0.188875384523
Take-out : -0.0590775725225
Price : 0.0804826394613


In [344]:
final_new_file=pd.concat([final_file, new_file["stars"]],axis=1)

In [345]:
final_new_file.columns

Index(['Noise', 'Kids', 'Wi-Fi', 'park', 'Take-out', 'Price', 'stars'], dtype='object')

In [346]:
final_new_file.corr(method="pearson")

,Noise,Kids,Wi-Fi,park,Take-out,Price,stars
Noise,1.000000,0.765354,0.052985,0.007578,-0.052597,0.057632,-0.144150
Kids,0.765354,1.000000,0.059883,0.121030,0.011953,0.052420,-0.056665
Wi-Fi,0.052985,0.059883,1.000000,0.062032,-0.008218,0.092396,0.029485
park,0.007578,0.121030,0.062032,1.000000,-0.006178,0.153497,0.188875
Take-out,-0.052597,0.011953,-0.008218,-0.006178,1.000000,-0.390985,-0.059078
Price,0.057632,0.052420,0.092396,0.153497,-0.390985,1.000000,0.080483
stars,-0.144150,-0.056665,0.029485,0.188875,-0.059078,0.080483,1.000000


## Mutiple linear regression

In [347]:
# We can use the linear model from sklearn package
from sklearn import linear_model

In [348]:
# Update X data frames (independent variables)
X = final_file[linear_features]
# y stays the same
y = new_file["stars"]

# A new linear regression model
regr = linear_model.LinearRegression()

# Fit X and y
regr.fit(X, y)

# Print out key parameters:
print ("coefficients: ")
print (regr.coef_)
print ("intercept: ")
print (regr.intercept_)
print ()

print("The fitted equation becomes:")
for i in range(0,len(linear_features)):
    print(round(regr.coef_[i],7),end='')
    print('*',end='')
    print('x',end='')
    print(i+1,end='')
    print(' ',end='')
    print('+',end='')
    print(' ',end='')
print('(',end='')
print(regr.intercept_,end='')
print(')')

print()
print('In which,')
for i in range(0,len(linear_features)):
    print('x',end='')
    print(i+1,end='')
    print(':',end='')
    print(linear_features[i])    

coefficients: 
[-0.24434186  0.14589935  0.02792024  0.2774254  -0.14099566  0.04104012]
intercept: 
3.49014790631

The fitted equation becomes:
-0.2443419*x1 + 0.1458994*x2 + 0.0279202*x3 + 0.2774254*x4 + -0.1409957*x5 + 0.0410401*x6 + (3.49014790631)

In which,
x1:Noise
x2:Kids
x3:Wi-Fi
x4:park
x5:Take-out
x6:Price


## Non-linear features selection by Random forest for Logistic regression

In [349]:
from sklearn.ensemble import RandomForestClassifier

In [350]:
features=["Noise","Wi-Fi","Price","Kids","Take-out","park"]
x=final_file[features]
y=new_file["stars"]

In [351]:
y.value_counts()

4.0    4794
3.5    4657
3.0    2755
4.5    1894
2.5    1307
2.0     504
5.0     178
1.5     166
1.0      18
Name: stars, dtype: int64

In [352]:
y=y>y.mean()
y.value_counts()

False    9407
True     6866
Name: stars, dtype: int64

In [355]:
rf_model=RandomForestClassifier(n_estimators=1000, max_features=2, oob_score=True)
rf_model=rf_model.fit(x,y)
rf_model

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=2, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=1, oob_score=True, random_state=None,
            verbose=0, warm_start=False)

In [356]:
rf_model.oob_score_

0.6011798684938241

In [357]:
for features,importance in zip(features, rf_model.feature_importances_):
    print (features, ":", importance)

Noise : 0.311738030194
Wi-Fi : 0.146255569373
Price : 0.2514452254
Kids : 0.0477860310017
Take-out : 0.0778992404227
park : 0.164875903609


In [358]:
final_file.columns

Index(['Noise', 'Kids', 'Wi-Fi', 'park', 'Take-out', 'Price'], dtype='object')

In [359]:
final_file["intercept"]=1
logit_features=["Noise","Wi-Fi","Price","park","intercept"]
x_logit=final_file[logit_features]
print (type(x_logit))
print (type(y))


<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


In [360]:
x_logit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16273 entries, 0 to 16272
Data columns (total 5 columns):
Noise        16273 non-null int64
Wi-Fi        16273 non-null int64
Price        16273 non-null float64
park         16273 non-null int64
intercept    16273 non-null int64
dtypes: float64(1), int64(4)
memory usage: 635.7 KB


In [361]:
import statsmodels.api as sm
logit=sm.Logit(y,x_logit)

In [362]:
# displaying the result of logistic regression
result=logit.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.667377
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                  stars   No. Observations:                16273
Model:                          Logit   Df Residuals:                    16268
Method:                           MLE   Df Model:                            4
Date:                Wed, 15 Mar 2017   Pseudo R-squ.:                 0.01987
Time:                        18:07:01   Log-Likelihood:                -10860.
converged:                       True   LL-Null:                       -11080.
                                        LLR p-value:                 5.391e-94
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Noise         -0.4591      0.029    -16.103      0.000        -0.515    -0.403
Wi-Fi          0.0881      0.

In [363]:
print (result.params)

Noise       -0.459114
Wi-Fi        0.088080
Price        0.111489
park         0.464414
intercept   -0.483475
dtype: float64


In [364]:
# odds ratio
print (np.exp(result.params))

# odds ratio is between 0 and 1: this feature has a negative correlation with "star rating"
# odds ratio is over 1: this feature has a positive correlation with "star rating"

Noise        0.631843
Wi-Fi        1.092075
Price        1.117942
park         1.591081
intercept    0.616637
dtype: float64
